In [ ]:
import pandas as pd
import os
import gc

# --- Paths for CTU-13 dataset ---
dataset_files = {
    '1': '1/capture20110810.binetflow',
    '2': '2/capture20110811.binetflow',
    '5': '5/capture20110815-2.binetflow',
    '9': '9/capture20110817.binetflow',
    '13': '13/capture20110815-3.binetflow'
}
# --- End paths ---

# Define the folder name to save sample results
output_folder = 'sampled_data'

# Create an empty dictionary to store all DataFrames
# This variable will 'live' after this cell is executed
dict_of_dataframes = {}

In [ ]:
print("Starting process: Reading data & taking samples...")

# 1. Create output folder if it doesn't exist
try:
    os.makedirs(output_folder, exist_ok=True)
    print(f"Output folder '{output_folder}' has been prepared.")
except Exception as e:
	print(f"Failed to create folder '{output_folder}': {e}")

# 2. Loop, read, save to dict, take samples, and save samples
for key, filepath in dataset_files.items():
    try:
        print(f"\n--- Processing Dataset {key} ---")
        print(f"Reading file: {filepath}...")
        
        # a. Read file
        df = pd.read_csv(filepath, low_memory=False)
        total_rows = len(df)
        
        # b. Save COMPLETE DataFrame to dictionary
        # This fulfills your request to access it later
        dict_of_dataframes[key] = df
        print(f"Dataset {key} ({total_rows} rows) has been loaded into memory.")

        # c. Determine sample size
        sample_size = min(100, total_rows)
        if sample_size < 100:
            print(f"Warning: File {key} only has {total_rows} rows. Taking {sample_size} rows.")
        else:
            print(f"Taking 100 random rows from file {key}...")

        # d. Take samples from the DataFrame that was just read
        sampled_df_botnet_not_spam = df[
            df['Label'].str.contains('botnet', case=False, na=False) &
            ~df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=3, random_state=42)

        sampled_df_botnet_and_spam = df[
            df['Label'].str.contains('botnet', case=False, na=False) &
            df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=4, random_state=42)

        sampled_df_no_botnet_no_spam = df[
            ~df['Label'].str.contains('botnet', case=False, na=False) &
            ~df['Label'].str.contains('spam', case=False, na=False)
        ].sample(n=4, random_state=42)

        # Combine the three DataFrames into one
        sampled_df = pd.concat([sampled_df_botnet_not_spam, sampled_df_botnet_and_spam, sampled_df_no_botnet_no_spam], ignore_index=True)

        # Shuffle the row order
        sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

        # Delete the three variables that have been combined to free memory
        del sampled_df_botnet_not_spam
        del sampled_df_botnet_and_spam
        del sampled_df_no_botnet_no_spam

        # Ensure variables are deleted and memory is more efficient
        gc.collect()

        # e. Determine output file name and save sample
        original_folder = os.path.dirname(filepath)  # Get original folder from file path
        folder_name = os.path.basename(original_folder)  # Get folder name like '1', '2', etc.
        
        # Create folder inside sampled_data with original folder name
        folder_output_path = os.path.join(output_folder, folder_name)
        os.makedirs(folder_output_path, exist_ok=True)

        base_filename = os.path.basename(filepath)  # Get original file name
        output_filepath = os.path.join(folder_output_path, base_filename)  # Combine with output folder
        
        sampled_df.to_csv(output_filepath, index=False)
        print(f"Success! Sample saved to '{output_filepath}'")
        
    except FileNotFoundError:
        print(f"WARNING: File not found at '{filepath}'. Dataset {key} will be skipped.")
    except Exception as e:
        print(f"Failed to process file {filepath}: {e}. Dataset {key} will be skipped.")

print("\n--- All processes completed ---")
print(f"Successfully loaded {len(dict_of_dataframes)} datasets into memory.")

Memulai proses: Membaca data & mengambil sampel...
Folder output 'sampled_data' telah disiapkan.

--- Memproses Dataset 1 ---
Membaca file: 1/capture20110810.binetflow...
Dataset 1 (100 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 1...
Sukses! Sampel disimpan ke 'sampled_data\1\capture20110810.binetflow'

--- Memproses Dataset 2 ---
Membaca file: 2/capture20110811.binetflow...
Dataset 2 (100 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 2...
Sukses! Sampel disimpan ke 'sampled_data\2\capture20110811.binetflow'

--- Memproses Dataset 5 ---
Membaca file: 5/capture20110815-2.binetflow...
Dataset 5 (100 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 5...
Sukses! Sampel disimpan ke 'sampled_data\5\capture20110815-2.binetflow'

--- Memproses Dataset 9 ---
Membaca file: 9/capture20110817.binetflow...
Dataset 9 (100 baris) telah dimuat ke memori.
Mengambil 100 baris acak dari file 9...
Sukses! Sampel disimpan ke 'sampled_data\9\captur

In [3]:
# Check which keys (datasets) are available in the dictionary
print(f"Datasets successfully loaded into memory: {list(dict_of_dataframes.keys())}")

# Example: Access and display the first 5 rows from dataset '5' (the COMPLETE one)
try:
    print("\nDisplaying .head() of dataset '5' (retrieved from memory):")
    
    # This is how you access the COMPLETE data
    df_5_full = dict_of_dataframes['5']
    
    display(df_5_full.head())
    print(f"Size of dataset '5' in memory: {df_5_full.shape}")
    
except KeyError:
    print("Dataset '5' was not successfully loaded (possibly error or not found).")

Dataset yang berhasil dimuat ke memori: ['1', '2', '5', '9', '13']

Menampilkan .head() dari dataset '5' (diambil dari memori):


,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/15 17:07:04.499651,0.382911,tcp,147.32.84.118,3387,->,178.211.170.158,6881,SRPA_FSRPA,0.0,0.0,11,1081,572,flow=Background-TCP-Established
1,2011/08/15 16:54:41.704838,0.000657,udp,221.4.204.82,41968,<->,147.32.84.229,13363,CON,0.0,0.0,2,563,78,flow=Background-UDP-Established
2,2011/08/15 16:44:00.127854,655.896484,udp,41.190.95.253,27025,<->,147.32.84.229,13363,CON,0.0,0.0,12,1690,1416,flow=Background-UDP-Established
3,2011/08/15 17:02:09.894977,7.832476,tcp,147.32.84.165,1528,->,205.188.186.137,587,FSPA_FSPA,0.0,0.0,59,7125,4142,flow=From-Botnet-V46-TCP-Established-SPAM
4,2011/08/15 17:01:37.243807,8.755043,tcp,147.32.84.165,1500,->,94.100.176.20,25,S_,0.0,NaN,3,186,186,flow=From-Botnet-V46-TCP-Attempt-SPAM


Ukuran dataset '5' di memori: (100, 15)
